In [ ]:
import fitz
import os

In [ ]:
import string
import re
from datetime import datetime
from pathlib import Path

# Maximum filename length for most filesystems
MAX_FILENAME_LENGTH = 255

def sanitize_filename(filename, max_length=MAX_FILENAME_LENGTH - 4):  # -4 for .pdf extension
    """
    Remove or replace invalid characters from filename.
    
    Security: Prevents path traversal and ensures filename safety.
    """
    # Remove invalid characters for filenames across Windows, Linux, and macOS
    invalid_chars = '<>:"/\\|?*'
    for char in invalid_chars:
        filename = filename.replace(char, '')
    
    # Remove control characters (ASCII 0-31)
    filename = ''.join(char for char in filename if ord(char) >= 32)
    
    # Remove leading/trailing whitespace and dots
    filename = filename.strip('. ')
    
    # Prevent path traversal attacks
    filename = filename.replace('..', '')
    filename = os.path.basename(filename)  # Remove any path components
    
    # Truncate to maximum length while trying to preserve meaning
    if len(filename) > max_length:
        filename = filename[:max_length].rsplit(' ', 1)[0]  # Try to break at word boundary
    
    return filename

def is_safe_path(base_path, target_path):
    """
    Verify that target_path is within base_path to prevent directory traversal.
    
    Security: Prevents accessing files outside intended directory.
    """
    try:
        base = Path(base_path).resolve()
        target = Path(target_path).resolve()
        return target.is_relative_to(base)
    except (ValueError, OSError):
        return False

def get_unique_filename(directory, base_name, extension=".pdf"):
    """
    Generate a unique filename by appending numbers if file exists.
    
    Security: Prevents race conditions and handles duplicates safely.
    """
    filepath = os.path.join(directory, base_name + extension)
    
    # Check for symlinks - security measure
    if os.path.islink(filepath):
        return None, "Target is a symbolic link"
    
    if not os.path.exists(filepath):
        return filepath, None
    
    # File exists, try numbered variants
    counter = 1
    max_attempts = 1000  # Prevent infinite loop
    
    while counter < max_attempts:
        # Calculate available space for counter
        suffix = f" ({counter})"
        max_base_length = MAX_FILENAME_LENGTH - len(extension) - len(suffix)
        truncated_base = base_name[:max_base_length] if len(base_name) > max_base_length else base_name
        
        new_name = f"{truncated_base}{suffix}{extension}"
        filepath = os.path.join(directory, new_name)
        
        if os.path.islink(filepath):
            counter += 1
            continue
            
        if not os.path.exists(filepath):
            return filepath, None
        
        counter += 1
    
    return None, "Too many files with same name"

# Configuration
print("=" * 80)
print("PDF BATCH RENAMER - Based on PDF Metadata Titles")
print("=" * 80)
print()

directory_path = input("Enter the directory path containing PDF files (or press Enter for current directory): ").strip()
if not directory_path:
    directory_path = "."

# Security: Validate directory path
try:
    directory_path = os.path.abspath(directory_path)
    if not os.path.exists(directory_path):
        print(f"\n❌ Error: Directory does not exist: {directory_path}")
        raise SystemExit(1)
    
    if not os.path.isdir(directory_path):
        print(f"\n❌ Error: Path is not a directory: {directory_path}")
        raise SystemExit(1)
    
    # Check if we have read/write permissions
    if not os.access(directory_path, os.R_OK | os.W_OK):
        print(f"\n❌ Error: Insufficient permissions for directory: {directory_path}")
        raise SystemExit(1)
        
except (OSError, PermissionError) as e:
    print(f"\n❌ Error accessing directory: {e}")
    raise SystemExit(1)

# Dry run option for safety
dry_run_input = input("Perform dry run first? (y/n, default: y): ").strip().lower()
dry_run = dry_run_input != 'n'

if dry_run:
    print("\n🔍 DRY RUN MODE - No files will be renamed\n")
else:
    print("\n⚠️  LIVE MODE - Files will be renamed\n")

auto_number = input("Automatically number duplicate filenames? (y/n, default: y): ").strip().lower()
handle_duplicates = auto_number != 'n'

print(f"\nScanning directory: {directory_path}")
print("-" * 80)

# Statistics
total_files = 0
renamed_count = 0
skipped_count = 0
error_count = 0
rename_operations = []  # Store operations for logging

# Create log filename with timestamp
log_filename = f"pdf_rename_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
log_path = os.path.join(directory_path, log_filename)

# Process files
for root_dir, cur_dir, files in os.walk(directory_path):
    for name in files:
        # Only process PDF files
        if not name.lower().endswith('.pdf'):
            continue
        
        total_files += 1
        file_path = os.path.join(root_dir, name)
        
        # Security: Verify path is within our base directory
        if not is_safe_path(directory_path, file_path):
            print(f"\n⚠️  SECURITY WARNING: Skipping file outside base directory: {file_path}")
            skipped_count += 1
            continue
        
        # Security: Skip symbolic links
        if os.path.islink(file_path):
            print(f"\n⚠️  SECURITY WARNING: Skipping symbolic link: {file_path}")
            skipped_count += 1
            continue
        
        print(f"\nProcessing: {file_path}")
        
        pdf = None
        try:
            # Open PDF and read metadata
            pdf = fitz.open(file_path)
            metadata = pdf.metadata
            
            if not metadata:
                print(f"  ⚠ Skipped: No metadata available")
                skipped_count += 1
                continue
            
            title = metadata.get("title", "").strip()
            
            # Check if title exists
            if not title:
                print(f"  ⚠ Skipped: No title in metadata")
                skipped_count += 1
                continue
            
            # Security: Limit title length to prevent abuse
            if len(title) > 1000:
                print(f"  ⚠ Skipped: Title too long (>1000 chars), possible metadata corruption")
                skipped_count += 1
                continue
            
            # Sanitize the title
            sanitized_title = sanitize_filename(title)
            
            if not sanitized_title:
                print(f"  ⚠ Skipped: Title contains only invalid characters")
                skipped_count += 1
                continue
            
            # Check minimum length
            if len(sanitized_title) < 1:
                print(f"  ⚠ Skipped: Sanitized title is empty")
                skipped_count += 1
                continue
            
            # Get unique filename (handles duplicates if enabled)
            if handle_duplicates:
                new_file_path, error_msg = get_unique_filename(root_dir, sanitized_title)
                if error_msg:
                    print(f"  ⚠ Skipped: {error_msg}")
                    skipped_count += 1
                    continue
            else:
                new_file_path = os.path.join(root_dir, sanitized_title + ".pdf")
            
            # Check if file already has the correct name
            if file_path == new_file_path:
                print(f"  ✓ Already named correctly")
                skipped_count += 1
                continue
            
            # Check if target file already exists (when not handling duplicates)
            if not handle_duplicates and os.path.exists(new_file_path):
                print(f"  ⚠ Skipped: File already exists with name '{os.path.basename(new_file_path)}'")
                skipped_count += 1
                continue
            
            # Close PDF before renaming
            pdf.close()
            pdf = None
            
            # Perform rename (or simulate in dry run)
            if not dry_run:
                # Security: Final check before rename
                if not is_safe_path(directory_path, new_file_path):
                    print(f"  ⚠️  SECURITY WARNING: Generated path outside base directory, skipping")
                    skipped_count += 1
                    continue
                
                os.rename(file_path, new_file_path)
                print(f"  ✓ Renamed to: {os.path.basename(new_file_path)}")
            else:
                print(f"  ✓ Would rename to: {os.path.basename(new_file_path)}")
            
            renamed_count += 1
            rename_operations.append({
                'original': file_path,
                'new': new_file_path,
                'title': title
            })
            
        except Exception as e:
            print(f"  ✗ Error: {str(e)}")
            error_count += 1
        
        finally:
            # Ensure PDF is closed
            if pdf is not None:
                try:
                    pdf.close()
                except:
                    pass

# Write log file
if rename_operations and not dry_run:
    try:
        with open(log_path, 'w', encoding='utf-8') as log_file:
            log_file.write(f"PDF Rename Log - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            log_file.write("=" * 80 + "\n\n")
            log_file.write(f"Directory: {directory_path}\n")
            log_file.write(f"Total files renamed: {renamed_count}\n\n")
            log_file.write("-" * 80 + "\n\n")
            
            for op in rename_operations:
                log_file.write(f"Original: {op['original']}\n")
                log_file.write(f"New:      {op['new']}\n")
                log_file.write(f"Title:    {op['title']}\n")
                log_file.write("-" * 80 + "\n")
        
        print(f"\n📝 Log file created: {log_filename}")
    except Exception as e:
        print(f"\n⚠️  Warning: Could not create log file: {e}")

# Print summary
print("\n" + "=" * 80)
print("SUMMARY")
print("=" * 80)
print(f"Total PDF files found: {total_files}")
if dry_run:
    print(f"Would rename: {renamed_count}")
else:
    print(f"Successfully renamed: {renamed_count}")
print(f"Skipped: {skipped_count}")
print(f"Errors: {error_count}")
print("=" * 80)

if dry_run and renamed_count > 0:
    print("\n💡 Tip: Run again with dry_run='n' to perform actual renaming")
elif not dry_run and renamed_count > 0:
    print("\n✅ Renaming completed successfully!")
    print(f"📋 Check the log file for details: {log_filename}")